### Packages & troubleshoot

In [1]:
!pip install transformers
!pip install accelerate
!pip install torch
!pip install torchvision torchaudio
!pip install gc
!pip install bitsandbytes

  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
  Using cached filelock-3.17.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached huggingface_hub-0.29.1-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)
Using cached huggingface_hub-0.29.1-py3-none-any.whl (468 kB)
Using cached regex-2024.11.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (792 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 10.4 MB/s eta 0:00:0000:01
Using cached tokenizers-0.21.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Using cached filelock-3.17.0-py3-none-any.whl (16 kB)
  Using cached accelerate-1.4.0-py3-none-any.whl.metadata (19 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64

In [1]:
import gc

import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import bitsandbytes

The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.


In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import re
import json
import tqdm
from tqdm import tqdm
# Enable tqdm for pandas
tqdm.pandas()

In [4]:
# Check if CUDA (GPU) is available
print(torch.cuda.is_available())  

True


In [5]:
#Memory usage handling
#Frees up unused GPU memory that was previously allocated by PyTorch.
torch.cuda.empty_cache()

#Triggers Python's garbage collector to reclaim unused objects in memory.
gc.collect()

40

### LLAMA Implementation

In [6]:
with open("access_token_llama.txt", "r") as file: 
    token = file.read() 
#os.environ["HUGGINGFACE_HUB_TOKEN"] = token

In [7]:
model_id = "meta-llama/Llama-3.3-70B-Instruct"

##### Other implementation

In [8]:
# model_id = "meta-llama/Llama-3.3-70B-Instruct"
# quantization_config_arg = BitsAndBytesConfig(load_in_4bit=True)

# pipeline = transformers.pipeline(
#     "text-generation",
#     model=model_id,
#     model_kwargs={"torch_dtype": torch.bfloat16},
#     device_map="auto",
#     quantization_config=quantization_config_arg,
#     token = token)

##### Model loading

In [9]:
quantization_config_arg = BitsAndBytesConfig(load_in_4bit=True)#,
                                            #llm_int8_enable_fp32_cpu_offload=True)

# Load the model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config_arg,
    torch_dtype=torch.bfloat16
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

In [10]:
model.hf_device_map

{'': 0}

#### Model implementation

##### Prompt build up

In [11]:
#General classification prompt {cliff_prompt}
with open("prompt/llama_cliff_prompt.txt", "r") as file: 
    cliff_prompt = file.read() 

#Codebook with topic and subtopic descriptions {topics_descrip}
with open("prompt/1_ordered_codebook_prompt.txt", "r") as file:  
    topics_descrip = file.read()

#Output examples {json_examples}
with open("prompt/output_examples_prompt.txt", "r") as file:  
    json_examples = file.read() 

# Trial texts {text}
#with open("prompt/nyt_prompt_example.txt", "r") as file: 
#    text = file.read()  

with open("prompt/knitting_prompt_example.txt", "r") as file:
    text = file.read()

In [12]:
text_type =  "New York Times"
#text_type =  "USA Congress"

full_cliff_prompt = cliff_prompt.format(text_type = text_type,
                                   json_examples = json_examples,
                                   topics_descrip = topics_descrip,
                                   text = text,
                                   year = "2024")

In [13]:
print(full_cliff_prompt)

<begin_of_text_token>You are a classifier for economic inequality-related texts based on a codebook. You will receive a text from the New York Times (year: 2024) that deals with economic inequality, and your task is to classify it according to the most prominent topic and subtopic. Use the codebook provided below.

# Instructions:
1. Determine the TOPIC:
Select the single primary topic (numbered 1 to 14) that best matches the text's main theme based on the descriptions provided below. Every text has a topic, choose the one that most closely aligns with the dominant message or has according keywords.
2. Determine the SUBTOPIC:
Select the corresponding primary subtopic (letter A to D). Use 'none' as subtopic if no subtopic aligns with the text within the chosen topic.
3. Justify:
Provide a justification of why you applied the selected code.

# JSON Output description:
- Respond with a JSON in the following fashion: 
{"topic": "number", "subtopic": "letter", "justification": "[insert 1-2 

##### Prompt trial

In [14]:
# Tokenize input and move to CUDA
inputs = tokenizer(full_cliff_prompt, return_tensors="pt").to("cuda")  # Move tensors to GPU

# Generate response
outputs = model.generate(**inputs, max_new_tokens=80,
                         pad_token_id=tokenizer.eos_token_id)

In [15]:
# Decode and print output
response_spec = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [16]:
response_spec[-500:]

'eedle so that the loops from the prior row can be pulled off the other needle without unraveling.\n\n# JSON Output: \n{"topic": "12", "subtopic": "A", "justification": "The text mentions the knitting industry\'s potential to pollute the environment, leading to greater economic inequalities for the global south, which relates to environmental degradation and its impact on economic inequality."} \nNote: The JSON Output provided is just an example, you should provide your own answer based on the text. \n'

##### Answer post-processing

In [17]:
#Regex explanation
#.: Matches any character except a newline (\n).
#*: Matches zero or more of the preceding character (in this case, any character).
#?: Makes the * non-greedy (or lazy), meaning it matches as few characters as 
#possible. Without the ?, the regex would try to match the longest possible 
#string, which would cause issues when dealing with multiple sets of curly braces.

In [18]:
def extract_last_json(text, idx, idx_error, column):
    # Find all occurrences of JSON-like (between keys) objects
    text_subset = text[-500:]
    matches = re.findall(r'\{.*?\}', text_subset.strip())
    json_error_keywords = ["number", "letter"]
    
    if matches:
        if not any(keyword in matches[-1] for keyword in json_error_keywords):
            return matches[-1]
        else:
            print(f"Warning: Error JSON in row {idx}")
            idx_error.append({"idx":idx, "error": "error_json", "column" : column})
            return text[-500:]  # Return the last match
    else:
        print(f"Warning: No JSON was found in row {idx}.")
        idx_error.append({"idx":idx, "error": "no_json", "column" : column})
        return text[-500:]

In [19]:
idx, idx_error = 0, []
json_result = extract_last_json(response_spec, idx, idx_error, "ordered")
print(json_result)

{"topic": "12", "subtopic": "A", "justification": "The text mentions the knitting industry's potential to pollute the environment, leading to greater economic inequalities for the global south, which relates to environmental degradation and its impact on economic inequality."}


##### Sampled dataset answers

In [20]:
# Dropped in server
# Path: "..\3_stm_fit_R\out_meta_stm.csv"
out_meta = pd.read_csv("out_meta_stm.csv")

In [21]:
out_meta.head(1)

,id_,author,year,date,year_n,source,dataset,text,tokens_R
0,0,MARC F. PLATTNER; Marc F. Plattner is on the s...,1980,1980-06-08,1,nyt,nyt,WHO GETS WHAT THE ZERO-SUM SOCIETY Distributi...,get zero sum society distribution possibilitie...


In [22]:
#General classification prompt {cliff_prompt}
with open("prompt/llama_cliff_prompt.txt", "r") as file: 
    cliff_prompt = file.read()

#Codebook with topic and subtopic descriptions {topics_descrip}
with open("prompt/1_ordered_codebook_prompt.txt", "r") as file:  
    ordered_topics_descrip = file.read() 
# with open("prompt/2_reversed_codebook_prompt.txt", "r") as file:  
#     reversed_topics_descrip = file.read() 
# with open("prompt/3_shuffled_codebook_prompt.txt", "r") as file:  
#     shuffled_topics_descrip = file.read() 
    
#Output examples {json_examples}
with open("prompt/output_examples_prompt.txt", "r") as file:
    json_examples = file.read() 

In [23]:
def cliff_case(row, cliff_prompt, json_examples, topics_descrip, idx, idx_error, column):
    text = row["text"]
    year = row["year"]
    text_type = "New York Times" if row["dataset"] == 'nyt' else "USA Congress"
    cliff_prompt = cliff_prompt.format(text_type = text_type,
                                       json_examples = json_examples,
                                       topics_descrip = topics_descrip,
                                       text = text,
                                       year = year)

    # Tokenize input and move to CUDA
    inputs = tokenizer(cliff_prompt, return_tensors="pt").to("cuda")  # Move tensors to GPU
    # Generate response
    outputs = model.generate(**inputs, 
                             max_new_tokens=90, 
                             pad_token_id=tokenizer.eos_token_id,
                             temperature=0.2)
    #Read outputs
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    #Extract last json
    json_result = extract_last_json(response, idx, idx_error, column)
    #{"justification": "Introduction of the INVEST Act, which focuses on helping small- and medium-sized businesses by allowing them to recover their costs faster.", "topic": "5", "subtopic": "A"} 

    return json_result

In [24]:
def llama_inference(df, cliff_prompt, json_examples):
    df_cliff = df.copy()  # Avoid modifying the original DataFrame
    idx_error = []
    df_cliff["llama_cliff_4bit_2502_ordered"] = df.progress_apply(lambda row: cliff_case(row,
                                                                       cliff_prompt, 
                                                                       json_examples, 
                                                                       ordered_topics_descrip,
                                                                       row.name, idx_error, column = "ordered"),
                                                                  axis=1)
    
    # df_cliff["llama_cliff_4bit_2402_reversed"] = df.progress_apply(lambda row: cliff_case(row,
    #                                                                    cliff_prompt, 
    #                                                                    json_examples, 
    #                                                                    reversed_topics_descrip,
    #                                                                    row.name, idx_error, column = "reversed"),
    #                                                               axis=1)
    
    # df_cliff["llama_cliff_4bit_2402_shuffled"] = df.progress_apply(lambda row: cliff_case(row,
    #                                                                    cliff_prompt, 
    #                                                                    json_examples, 
    #                                                                    shuffled_topics_descrip,
    #                                                                    row.name, idx_error, column = "shuffled"),
    #                                                               axis=1)

    #def most_common_result(row):
    #    results = [{"topic": r["topic"], "subtopic": r["subtopic"]} for r in [row["llama_cliff_4bit_2402_ordered"], row["llama_cliff_4bit_2402_reversed"], row["llama_cliff_4bit_2402_shuffled"]] if r is not None]
    #    if results:
    #        most_common = Counter(map(tuple, results)).most_common(1)
    #        return dict(most_common[0][0]) if most_common else {"topic": row["llama_cliff_4bit_2402_ordered"]["topic"], "subtopic": row["llama_cliff_4bit_2402_ordered"]["subtopic"]}
    #    return {"topic": row["llama_cliff_4bit_2402_ordered"]["topic"], "subtopic": row["llama_cliff_4bit_2402_ordered"]["subtopic"]}
    
    #df_cliff["llama_cliff_4bit_2402_most_common"] = df_cliff.apply(most_common_result, axis=1)
    
    idx_error_df = pd.DataFrame(idx_error)

    
    idx_error_df = pd.DataFrame(idx_error)
    
    return df_cliff, idx_error_df

In [25]:
df1 = out_meta.sample(n=10, axis=0)

In [26]:
df2, idx_error_df = llama_inference(df1, cliff_prompt,json_examples)

100%|██████████| 10/10 [00:58<00:00,  5.85s/it]


In [27]:
for i,text in enumerate(df2["llama_cliff_4bit_2502_ordered"]):
    print(i, text)

0 {"topic": "2", "subtopic": "B", "justification": "The text discusses tax provisions, specifically the leasing of unused tax benefits, and its implications on corporate income tax, which falls under the topic of Macroeconomics and the subtopic of Taxation."}
1 {"topic": "6", "subtopic": "A", "justification": "The text focuses on introducing legislation to correct a longtime inequity in military compensation, specifically providing separation pay benefits for enlisted personnel who are involuntarily separated from the service, which falls under public sector employment and NYC policies, particularly public employment salaries."}
2 {"topic": "4", "subtopic": "A", "justification": "The text discusses the conversion of existing rental apartments to condominium or cooperative ownership, focusing on the benefits and drawbacks of this trend, including tax advantages, appreciation, land use, preservation of housing stock, conservation, participatory democracy, and civic responsibility."}
3 {"

In [28]:
idx_error_df

""


##### Dataset run

In [30]:
chunk_size = 2000
num_chunks = len(out_meta) // chunk_size + (1 if len(out_meta) % chunk_size != 0 else 0)

In [ ]:
output_folder = "llama_cliffs"

for i in range(num_chunks):
    # Define the chunk range
    start_idx = i * chunk_size
    end_idx = min(start_idx + chunk_size, len(out_meta))
    out_meta_chunk = out_meta[start_idx:end_idx] 
    print(f"Processing chunk {i+1} ({start_idx}:{end_idx})")

    # Run inference
    chunk_cliff, chunk_error_idx = llama_inference(out_meta_chunk, cliff_prompt, json_examples)

    # Save individual chunk results in the "llama_cliffs" folder
    chunk_cliff.to_csv(os.path.join(output_folder, f'out_meta_chunk_{i+1}_cliff_2502.csv'), index=False)
    chunk_error_idx.to_pickle(os.path.join(output_folder, f'idx_error_chunk_{i+1}_cliff_2502.pkl'))

Processing chunk 1 (0:2000)


  4%|▎         | 72/2000 [06:12<3:48:12,  7.10s/it]

  6%|▌         | 113/2000 [09:43<2:44:45,  5.24s/it]

 14%|█▍        | 279/2000 [24:04<2:51:50,  5.99s/it]

 15%|█▍        | 293/2000 [25:31<4:04:18,  8.59s/it]

...

  9%|▉         | 179/2000 [16:19<3:23:12,  6.70s/it]

 11%|█▏        | 225/2000 [20:38<3:12:15,  6.50s/it]

 16%|█▌        | 312/2000 [28:37<2:37:37,  5.60s/it]

 18%|█▊        | 363/2000 [33:24<2:36:55,  5.75s/it]

 20%|██        | 407/2000 [37:22<2:12:56,  5.01s/it]

In [ ]:
for i in range(20):
    print(i, chunk_cliff.iloc[i]["llama_cliff_4bit_2502_ordered"])

##### Import of cliff dfs

In [4]:
output_folder = "llama_cliffs"

# Initialize lists to store loaded data
all_cliff_dfs = []  # To store DataFrames
all_error_indices = []  # To store error indices

# Get all file names in the output folder
for i in range(9):
    # Load DataFrame and append to the list
    cliff_file = os.path.join(output_folder, f'out_meta_chunk_{i+1}_cliff_2502.csv')
    error_file = os.path.join(output_folder, f'idx_error_chunk_{i+1}_cliff_2502.pkl')

    if os.path.exists(cliff_file):
        all_cliff_dfs.append(pd.read_csv(cliff_file))
    if os.path.exists(error_file):
        all_error_indices.append(pd.read_pickle(error_file))

In [5]:
# Merge all the DataFrames and lists
llama_cliff = pd.concat(all_cliff_dfs, ignore_index=True)  # Combine DataFrames
llama_error_indices =  pd.concat(all_error_indices, ignore_index=True) # Flatten error indices

In [6]:
len(llama_cliff)
len(llama_error_indices)

298

In [7]:
llama_error_indices[llama_error_indices["error"] == "no_json"]

,idx,error,column
0,70,no_json,ordered
1,111,no_json,ordered
3,291,no_json,ordered
4,334,no_json,ordered
8,471,no_json,ordered
...,...,...,...
285,12618,no_json,ordered
286,12628,no_json,ordered
287,12814,no_json,ordered
288,12871,no_json,ordered


In [39]:
llama_cliff["llama_cliff_4bit_2502_ordered"][13757]

'{"topic": "9", "subtopic": "A", "justification": "The text focuses on the Medicare Physician-Payment Equity Act, which aims to address geographic disparities in physician payment, particularly in rural areas, and improve access to healthcare in these regions."}'

In [41]:
# Save the merged results
llama_cliff.to_csv('llama_cliff_merged_2502.csv', index=False)
llama_error_indices.to_csv('llama_errors_merged_2502.csv', index=False)

In [9]:
llama_cliff.tail()

,id_,author,year,date,year_n,source,dataset,text,tokens_R,llama_cliff_4bit_2502_ordered
14453,CREC-2024-12-11-pt1-PgS6963-4-1,Mr. SANDERS,2024,2024-12-11,45,Independent,Congress,"Mr. President, as the holiday season approache...",president holiday season approach appropriate ...,"{""topic"": ""7"", ""subtopic"": ""A"", ""justification..."
14454,CREC-2024-12-12-pt1-PgH7130-2,Mr. BOWMAN,2024,2024-12-12,45,Democrat,Congress,"Mr. Speaker, I thank my sister Cori Bush for y...",speaker thank sister cori bush yield speaker w...,"{""topic"": ""7"", ""subtopic"": ""A"", ""justification..."
14455,CREC-2024-12-12-pt1-PgH7130-6,Ms. BUSH,2024,2024-12-12,45,Democrat,Congress,"Mr. Speaker, Congresswoman Ramirez came into C...",speaker congresswoman ramirez come congress li...,"{""topic"": ""1"", ""subtopic"": ""B"", ""justification..."
14456,CREC-2024-12-16-pt1-PgH7151-10,Mr. STEIL,2024,2024-12-16,45,Republican,Congress,"Mr. Speaker, I yield myself such time as I may...",speaker yield time consume speaker measure hal...,"{""topic"": ""6"", ""subtopic"": ""A"", ""justification..."
14457,CREC-2024-12-20-pt1-PgS7321-0,Mr. SANDERS,2024,2024-12-20,45,Independent,Congress,"Mr. President, I rise today to honor Hugh Espe...",president rise today honor hugh espey iowa cit...,"{""topic"": ""7"", ""subtopic"": ""A"", ""justification..."
